In [2]:
import json
import glob
from txtai.embeddings import Embeddings

In [3]:
files = glob.glob(f"../data/data_saha/*/*/*.json")
print (len(files))

2460


In [56]:
documents = []
segments = {"files": []}
x=0
for filename in files:
    with open(filename, "r") as f:
        data = json.load(f)
    testimony = data["testimony"]
    # segments["files"].append(filename)
    for i, segment in enumerate(testimony):
        speaker, dialogue, gender = segment
        # segments[x] = [filename, i]
        # segments["segments"].append({i})
        documents.append((f"{filename}_{i}---{speaker}_{gender}", dialogue, speaker, gender))
        x=x+1
print (len(documents))

675940


In [57]:
documents[:1]

[('../data/data_saha\\amnesty_hearings\\aliwal_north\\54583.json_0---MR PRIOR_Male',
  'May I proceed Mr Chairman?',
  'MR PRIOR',
  'Male')]

In [58]:
embeddings = Embeddings({"path": "sentence-transformers/nli-mpnet-base-v2", "content": True})

In [59]:
embeddings.index([(uid, text, f"{speaker} {gender}") for uid, text, speaker, gender in documents[:1000]])

In [63]:
embeddings.explain("violence by guns", limit=1)

[{'id': '../data/data_saha\\amnesty_hearings\\aliwal_north\\54583.json_263---PHILA MARTIN DOLO_Male',
  'text': 'There were guns.',
  'score': 0.7930391430854797,
  'tokens': [('There', -0.005476236343383789),
   ('were', -0.012428700923919678),
   ('guns.', 0.5534553527832031)]}]

In [68]:
from IPython.display import HTML

def plot(query):
    result = embeddings.explain(query, limit=1)[0]
    
    output = f"<b>{query}</b><br/>"
    spans = []
    for token, score in result["tokens"]:
        color = None
        if score >= 0.1:
            color = "#fdd835"
        elif score >= 0.075:
            color = "#ffeb3b"
        elif score >= 0.05:
            color = "#ffee58"
        elif score >= 0.02:
            color = "#fff59d"

        spans.append((token, score, color))

    if result["score"] >= 0.05 and not [color for _, _, color in spans if color]:
        mscore = max([score for _, score, _ in spans])
        spans = [(token, score, "#fff59d" if score == mscore else color) for token, score, color in spans]

    for token, _, color in spans:
        if color:
            output += f"<span style='background-color: {color}'>{token}</span> "
        else:
            output += f"{token} "

    return output

HTML(plot("burn"))